# Install dependencies in a virtual environment

Install packages for efficiently parsing CSV files.

In [ ]:
!pip install casanova

Install packages for NLP analysis.

In [ ]:
!pip install bertopic hdbscan nltk sentence_transformers scikit-learn umap

Also install `protobuf` version 3.20 because `SentenceTransformer` needs it.

In [ ]:
!pip install protobuf==3.20.0

Install tools for visualizing the results.

In [ ]:
!pip install networkx ipysigma plotly install pelote

# Prepare data

We're going to be taking advantage of transformers and word-embedding, which in theory do not require pre-processing sentences that are sent to the topic model. Nevertheless, some pre-processing can be useful (i.e. removing HTML tags). In our case, we will want to remove the phrase "Il faut" at the start of every proposition because it is not meaningful in the context of our corpus.

In [1]:
DATAFILE = "mieux-sinformer.csv" # change this according to file path

In [2]:
def special_preprocessing(string):
    string = string.lower()
    bow = string.split()
    if bow[:2] == ["il", "faut"]:
        bow = bow[2:]
    return " ".join(bow)

With that helper function in place, we'll parse our data file and create a list of our documents. (list of strings)

In [3]:
import casanova

with open(DATAFILE) as f:
    reader = casanova.reader(f)
    docs = [special_preprocessing(cell) for cell in reader.cells(column="Proposition")]
    print(f"Dataset includes {len(docs)} docs.")

Dataset includes 1723 docs.


We will also need to prepare a list of stop words, which will be excluded from the topics' representations.

In [4]:
from nltk.corpus import stopwords

stoplist = stopwords.words("french")
ADDITIONAL_STOPWORDS = ["plus", "chaque", "tout", "tous", "toutes", "toute", "leur", "leurs", "comme", "afin", "pendant", "lorsque"]
stoplist.extend(ADDITIONAL_STOPWORDS)

# Vectorize the data

### Word embeddings
The first step is to transform a string (sentence) into an array of numbers (vector) or in other words, to "vectorize" the text. There are many ways to do this. By default, `BERTopic`'s privileges word embeddings, which are currently the most powerful way to encode a document because they represent words' relationships to other words in the context of a sentence.

However, `BERTopic`'s default sentence transformer was trained on English-language texts. Because we are working with a corpus of sentences exclusively in French, we want to take advantage of a language model trained on French-language texts, such as the `CamemBERT` model. Fortunately, a data scientist at *La Javaness* Van Tuan Dang ("dangvantuan" on HuggingFace) has trained and published a sentence transformer model based on the `CamemBERT` base model. We will simply import this pre-trained transformer.

In [5]:
from sentence_transformers import SentenceTransformer

sentences = ['Bonjour tout le monde.', 'Ça va ?']

camembert_sentence_transformer = SentenceTransformer("dangvantuan/sentence-camembert-large")
example_embeddings = camembert_sentence_transformer.encode(sentences, show_progress_bar=True)

print(f"Sentence 1 has {len(sentences[0])} characters, and the embedding is {len(example_embeddings[0])} long.")
print(f"Sentence 2 has {len(sentences[1])} characters, and the embedding is {len(example_embeddings[1])} long.")

No sentence-transformers model found with name /Users/kelly.christensen/.cache/torch/sentence_transformers/dangvantuan_sentence-camembert-large. Creating a new one with MEAN pooling.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Sentence 1 has 22 characters, and the embedding is 1024 long.
Sentence 2 has 7 characters, and the embedding is 1024 long.


As seen in the cell above, the embeddings created by Dang's sentence transformer `sentence-camembert-large` have the same length, despite the fact that the sentences the arrays represent are different lenghts. The embeddings have the same length because, rather than directly representing words as numbers (aka bag of words), the transformer creates a rich representation that takes into account 1024 dimensions.

### Construct the topic model with the desired parameters

Finally, having decided on an effective embedding model (very important!) and produced stop words for the topics' representations, we're ready to assemble the topic model.

In [6]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

stoplist = stopwords.words("french")
ADDITIONAL_STOPWORDS = ["plus", "chaque", "tout", "tous", "toutes", "toute", "leur", "leurs", "comme", "afin", "pendant", "lorsque"]

def set_model_parameters(embedding_model):

    # Step 1 - Extract embeddings
    embedding_model = embedding_model

    # Step 2 - Reduce dimensionality
    umap_model = UMAP(angular_rp_forest=True, metric='cosine', n_components=10, n_neighbors=30, min_dist=0.1)

    # Step 3 - Cluster reduced embeddings
    hdbscan_model = HDBSCAN(min_cluster_size=13, min_samples=3, prediction_data=True, metric='euclidean', cluster_selection_method='eom')

    # Step 4 - Tokenize topics
    stoplist.extend(ADDITIONAL_STOPWORDS)
    vectorizer_model = CountVectorizer(stop_words=stoplist, ngram_range=(1, 3))

    # Step 5 - Create topic representation
    ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True, bm25_weighting=True)

    # Topic model
    return BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        ctfidf_model=ctfidf_model,
        diversity=0.5,
        n_gram_range=(1,3),
        nr_topics='auto',
    )

# Fit the topic model

Download the pre-trained embedding model from `SentenceTransformers` and attribute it to the variable `embedding model`. Then use that model to encode all the documents in our corpus.

In [7]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("dangvantuan/sentence-camembert-large")
embeddings = camembert_sentence_transformer.encode(docs, show_progress_bar=True)

No sentence-transformers model found with name /Users/kelly.christensen/.cache/torch/sentence_transformers/dangvantuan_sentence-camembert-large. Creating a new one with MEAN pooling.


Batches:   0%|          | 0/54 [00:00<?, ?it/s]

Create an instance of the model with all the parameters defined and our selected embedding model.

In [8]:
topic_model = set_model_parameters(embedding_model)

Fit the model to the documents in our corpus and to the embeddings we prepared.

In [9]:
topic_model.fit(docs, embeddings)

topic_model.get_topic_info()

,Topic,Count,Name
0,-1,465,-1_sans_citoyens_médias_donner
1,0,325,0_esprit_experts_information_politiques
2,1,249,1_financement_concentration médias_milliardair...
3,2,226,2_fake news_france_chaînes_tv
4,3,112,3_algorithmes_facebook_twitter_identité
5,4,81,4_médias information_information_enseignement_...
6,5,57,5_journaux_rendre accessible_papier_netflix
7,6,46,6_charte munich_déontologie journalistique_pro...
8,7,40,7_sensibiliser_réseaux sociaux former_images_i...
9,8,38,8_arrêtent_journalistes restent_fact checking ...


### Explore the model's predictions

In [20]:
# optional
topic_model = BERTopic.load('best_bertopic.model')

In [21]:
hierarchical_topics = topic_model.hierarchical_topics(docs)

tree = topic_model.get_topic_tree(hierarchical_topics)
print(tree)

100%|███████████████████████████████████████████| 14/14 [00:06<00:00,  2.14it/s]

.
├─anonymat réseaux sociaux_identité_réseaux sociaux responsables_supprimer_diffusent
│    ├─■──réseaux sociaux responsables_diffusent_détenteurs_responsables contenus_sociaux considérés ── Topic: 12
│    └─■──anonymat réseaux sociaux_réseaux sociaux interdire_supprimer_vrai nom_bannir ── Topic: 10
└─journalistes_éducation médias_école_news_fake
     ├─partiaux_orientation politique_tendance_neutralité_lfi
     │    ├─orientation politique_tendance_neutralité_affiliations_lfi
     │    │    ├─■──laissent_journalistes restent_humilité_neutres_propagande ── Topic: 9
     │    │    └─■──gauche_lfi_orienté_partiaux_politique information ── Topic: 7
     │    └─■──laisser liberté_liberté presse_influence_gouvernement_médias annoncent mouvance ── Topic: 11
     └─presse_éducation médias_école_créer_fake news
          ├─éducation médias_école_presse_public_fake news
          │    ├─■──charte munich_créer_conseil_code déontologie_déontologie médias ── Topic: 6
          │    └─éducation méd

In [22]:
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics).show(renderer='iframe_connected')

In [23]:
topic_model.visualize_barchart().show(renderer='iframe_connected')

In [24]:
topic_model.visualize_heatmap(n_clusters=4).show(renderer='iframe_connected')

In [25]:
topic_model.visualize_term_rank(log_scale=True).show(renderer='iframe_connected')

In [26]:
topic_model.visualize_topics().show(renderer='iframe_connected')

In [27]:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings).show(renderer='iframe_connected')

Update the topic labels.

In [28]:
topic_labels = topic_model.generate_topic_labels(nr_words=3,
                                topic_prefix=True,
                                separator=" - ")

topic_model.set_topic_labels(topic_labels)

Print the results to a CSV file.

In [29]:
import casanova

PREDICTIONS_FILE = 'bertopic_topics_notebook.csv'

results = topic_model.get_document_info(docs=docs)
results.to_csv()
with open(DATAFILE) as f, open(PREDICTIONS_FILE, 'w') as of:
    enricher = casanova.enricher(f, of, add=["document", "topic", "name", "top_n_words", "probability", "representative_document"])
    for i, row in enumerate(enricher):
        enricher.writerow(row=row, add=[results["Document"][i], results["Topic"][i], results["Name"][i], results["Top_n_words"][i], results["Probability"][i], results["Representative_document"][i]])

# Visualize the results

In a dictionary, index the documents and their metadata by each document's Id.

In [30]:
import csv

with open(PREDICTIONS_FILE) as f:
    reader = csv.DictReader(f)
    for row in reader:
        doc_index = {row["Id"]:row for row in reader}

While parsing the matrix file, create nodes and edges and label them with data from the topic predictions.

In [31]:
MATRIX_FILE = 'defacto_covotes.csv'

In [32]:
import networkx as nx
from statistics import mean
import casanova

with open(MATRIX_FILE) as f:
    reader = casanova.reader(f)
    pid1_pos = reader.headers['pid1']
    pid2_pos = reader.headers['pid2']
    vote1_pos = reader.headers['vote1']
    vote2_pos = reader.headers['vote2']
    count_pos = reader.headers['count']
    
    G = nx.Graph()
    for row in reader:
        pid1 = row[pid1_pos]
        pid2 = row[pid2_pos]

        # If the matrix refers to documents not in the original data, skip 
        if not doc_index.get(pid1) or not doc_index.get(pid2):
            continue

        # Base an edge's weight on the support that two related propositions received
        average_nb_votes = mean([int(doc_index[pid1]["Nb de votes"]), int(doc_index[pid2]["Nb de votes"])])
        weight = int(row[count_pos])/average_nb_votes

        # Unless already added to the Graph, add both nodes in the matrix row and create an edge between them
        print(doc_index[pid1])
        if not G.has_node(pid1) and not str(doc_index[pid1]["topic"]) == "-1":
            G.add_node(pid1, label=doc_index[pid1]["Proposition"], **doc_index[pid1])

        if not G.has_node(pid2) and not str(doc_index[pid2]["topic"]) == "-1":
            G.add_node(pid2, label=doc_index[pid2]["Proposition"], **doc_index[pid2])

        if row[vote1_pos] == row[vote2_pos]:  # To-do: test if we can try all 3 types of cases
            G.add_edge(pid1, pid2, weight=weight)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
import pelote

GEFX_FILE = 'sparsification.gexf'

H = pelote.multiscale_backbone(G, alpha=0.05)
nx.write_gexf(H, GEFX_FILE)

In [34]:
import networkx as nx
from ipysigma import Sigma

In [35]:
# Importing a gexf graph
g = nx.read_gexf(GEFX_FILE)

In [36]:
# Displaying the graph with a size mapped on degree and
# a color mapped on a categorical attribute of the nodes
Sigma(g, node_size=g.degree, node_color='topic')

Sigma(nx.Graph with 1,722 nodes and 30,552 edges)